In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [4]:
X.shape

(768, 8)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [8]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6538 - loss: 0.6848 - val_accuracy: 0.7208 - val_loss: 0.6412
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6899 - loss: 0.6130 - val_accuracy: 0.7403 - val_loss: 0.5999
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7109 - loss: 0.5894 - val_accuracy: 0.7468 - val_loss: 0.5660
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7695 - loss: 0.5493 - val_accuracy: 0.7597 - val_loss: 0.5389
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7523 - loss: 0.5279 - val_accuracy: 0.7662 - val_loss: 0.5194
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7221 - loss: 0.5407 - val_accuracy: 0.7922 - val_loss: 0.5034
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7646 - loss: 0.4924 - val_accuracy: 0.7792 - val_loss: 0.4903
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7957 - loss: 0.4793 - val_accuracy: 0.7857 - 

In [10]:
'''
now, we are going to be setting hyperparameters in such a way that we can get the best accuracy
what hyperparameters are to be set here?
1. selecting appropriate optimizer
2. number of nodes in each layer
3. number of hidden layers

NOTE:- instead of manually selecting these values by doing arduous trial and error method, let us automate the whole process of selecting the best hyperparameter.
'''

'\nnow, we are going to be setting hyperparameters in such a way that we can get the best accuracy\nwhat hyperparameters are to be set here?\n1. selecting appropriate optimizer\n2. number of nodes in each layer\n3. number of hidden layers\n\nNOTE:- instead of manually selecting these values by doing arduous trial and error method, let us automate the whole process of selecting the best hyperparameter.\n'

In [11]:
pip install -U keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [12]:
import kerastuner as kt
#library used for automated hyperparameter tuning.

<ipython-input-12-9e1f9cd794ca>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [13]:
#function for selecting the best suitable OPTIMIZER
def build_model (hp):
  #first make the NN
  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim = 8))
  model.add(Dense(1,activation='sigmoid'))

  #we gave it 3 choices, you can give as many as you want out of all the available optimizers.
  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [14]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)
'''
so, there will be 5 trials, and kt will randomly make a choice and use it as the optimizer for model training and evaluation.
'''

'\nso, there will be 5 trials, and kt will randomly make a choice and use it as the optimizer for model training and evaluation.\n'

In [15]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
'''
for each trial, we have 5 epochs.
'''

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 15s


'\nfor each trial, we have 5 epochs.\n'

In [16]:
tuner.get_best_hyperparameters()[0].values
'''
when we use get_best_hyperparameters, all the 5 models get arranged in the decreasing order of val_accuracy score as that's what we defined in the objective.
we want to get the best model out of the 5, which is the first model. so, we used [0]. Now, we used .values for getting the best optimizer.
'''

"\nwhen we use get_best_hyperparameters, all the 5 models get arranged in the decreasing order of val_accuracy score as that's what we defined in the objective.\nwe want to get the best model out of the 5, which is the first model. so, we used [0]. Now, we used .values for getting the best optimizer.\n"

In [17]:
model = tuner.get_best_models(num_models=1)[0]
'''
we trained all the 5 models for 5 epochs, so why not use the best model for further training. hence, we extracted the best model.
so, because we just need one out of the 5 models we trained, we used num_models=1 which gives us one model and we do [0] so
that it gives us the best one model. this is because get_best_models sets all the models from best to worst. so, [0] gives us the
best model.
'''

'\nwe trained all the 5 models for 5 epochs, so why not use the best model for further training. hence, we extracted the best model.\n'

In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))
'''
as this best model was already trained for 5 epohcs, we keep the initial epoch to be 6.
'''

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.7005 - loss: 0.6070 - val_accuracy: 0.7532 - val_loss: 0.5899
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7327 - loss: 0.5760 - val_accuracy: 0.7532 - val_loss: 0.5760
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7335 - loss: 0.5718 - val_accuracy: 0.7597 - val_loss: 0.5650
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7285 - loss: 0.5679 - val_accuracy: 0.7597 - val_loss: 0.5551
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7388 - loss: 0.5658 - val_accuracy: 0.7662 - val_loss: 0.5474
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7384 - loss: 0.5509 - val_accuracy: 0.7727 - val_loss: 0.5402
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7612 - loss: 0.5331 - val_accuracy: 0.7792 - val_loss: 0.5342
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7483 - loss: 0.5264 - val_accuracy: 0.77

'\nas this best model was already trained for 5 epohcs, we keep the initial epoch to be 6.\n'

In [20]:
#setting the NUMBER OF NODES in a layer.
def build_model(hp):
  model = Sequential()
  #step = 8 means that as we are doing 5 trials, and as the min value of the number of nodes is 8 and the max being 128, and also
  #considering the point that the number of nodes are to be taken randomly, this random number will be nothing but the multiple
  #of 8. so, it could be 24, 72, 120, and so on.
  units = hp.Int('units', min_value = 8, max_value = 128, step = 8)
  model.add(Dense(units=units, activation = 'relu', input_dim = 8))
  model.add(Dense(1,activation='sigmoid'))

  #as before, we got the suitable optimizer as adam, hence here too we will be taking adam as the optimizer.
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [33]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory = 'mydir', project_name ='Dev')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 15s


In [50]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [39]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7374 - loss: 0.5310 - val_accuracy: 0.7727 - val_loss: 0.5127
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7744 - loss: 0.4806 - val_accuracy: 0.7857 - val_loss: 0.4951
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7574 - loss: 0.5030 - val_accuracy: 0.7922 - val_loss: 0.4864
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7911 - loss: 0.4759 - val_accuracy: 0.7922 - val_loss: 0.4824
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7742 - loss: 0.4753 - val_accuracy: 0.7857 - val_loss: 0.4780
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7842 - loss: 0.4599 - val_accuracy: 0.7922 - val_loss: 0.4732
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7833 - loss: 0.4632 - val_accuracy: 0.7922 - val_loss: 0.4715
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7657 - loss: 0.4767 - val_accuracy: 0.79

In [40]:
#setting the NUMBER OF LAYERS in the neural network.
def build_model(hp):

  model = Sequential()
  model.add(Dense(72, activation = 'relu', input_dim = 8))
  '''
  here, the num_layers (number of layers) will be starting from 1 to 10. so, say num_layers = 5, the loop iterates 5 times, giving us
  5 hidden layers in the neural network.
  NOTE:- num_layers will be assigned integer randomly from 1 to 10. so, as 5 trials, there will be 5 randoms numbers assigned to the
  num_layers, out of which one giving the best val_accuracy score will get selected.
  '''
  for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):
    model.add(Dense(72, activation = 'relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model


In [41]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory = 'mydir', project_name ='Dev1')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 28s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [47]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [48]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7519 - loss: 0.4705 - val_accuracy: 0.8312 - val_loss: 0.4590
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7998 - loss: 0.4237 - val_accuracy: 0.8182 - val_loss: 0.4579
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7908 - loss: 0.4398 - val_accuracy: 0.7987 - val_loss: 0.4601
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7912 - loss: 0.4378 - val_accuracy: 0.8052 - val_loss: 0.4611
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8140 - loss: 0.4014 - val_accuracy: 0.7987 - val_loss: 0.4645
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8211 - loss: 0.4186 - val_accuracy: 0.8117 - val_loss: 0.4574
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7943 - loss: 0.4076 - val_accuracy: 0.8117 - val_loss: 0.4673
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7966 - loss: 0.3957 - val_accuracy: 0